# GeoJSON polígons
Codi per generar els GeoJSON de les vegueries i les comarques rellevants per al repte.

In [1]:
import json
import geopandas as gpd
import pandas as pd
import numpy as np

In [161]:
## Nombre de comarques que surten al repte

TOTAL_COMARQUES = ['Alt Camp', 'Noguera', 'Baix Camp', 'Barcelonès', 'Priorat', 'Baix Ebre', 'Segrià', "Ribera d'Ebre", "Pla d'Urgell", 'Alt Penedès', 'Gironès', 'Vallès Occidental', 'Terra Alta', 'Osona', 'Baix Penedès', 'Montsià', 'Pallars Jussà', 'Baix Llobregat', 'Bages', "Pla de l'Estany", 'Anoia', 'Garrigues', 'Alt Empordà', 'Alt Urgell', 'Vallès Oriental', 'Segarra', 'Moianès', 'Vallespir', 'Ripollès', 'Berguedà', 'Maresme', 'Cerdanya', 'Alta Cerdanya', 'Tarragonès', 'Garrotxa', 'Alta Ribagorça', 'Solsonès', "Aran", 'Conflent', 'Andorra', 'Garraf', 'Baix Empordà', 'Conca de Barberà', 'Rosselló', 'Urgell', 'Selva', 'Pallars Sobirà', 'Capcir']

## Importació del GeoJSON de municipis
S'eliminen les columnes que no interessen.

In [27]:
df = gpd.read_file("municipis_Cat_And_CN_Franja.geojson").drop(columns=["extrude", "cartodb_id", "visibility", "codi", "municipi", "OGR", "OGR_STYLE", "tessellate"])
df

,vegueria,territori,comarca,geometry
0,Ponent,CAT,Garrigues,"POLYGON ((0.88503 41.44265, 0.88726 41.43778, ..."
1,Ponent,CAT,Garrigues,"POLYGON ((0.95157 41.35193, 0.95232 41.35767, ..."
2,Ponent,CAT,Garrigues,"POLYGON ((0.92870 41.46173, 0.91468 41.46485, ..."
3,Ponent,CAT,Garrigues,"POLYGON ((0.81071 41.51845, 0.80361 41.52047, ..."
4,Ponent,CAT,Garrigues,"POLYGON ((0.76766 41.33399, 0.74101 41.35421, ..."
...,...,...,...,...
1320,Ponent,FRANJA,La Llitera,"POLYGON ((0.35647 41.89048, 0.35652 41.89074, ..."
1321,Ponent,FRANJA,La Llitera,"POLYGON ((0.31412 41.68997, 0.31417 41.69070, ..."
1322,Ponent,FRANJA,La Llitera,"POLYGON ((0.31244 42.06176, 0.31409 42.06371, ..."
1323,Ponent,FRANJA,La Llitera,"POLYGON ((0.50910 41.91186, 0.50969 41.91215, ..."


## GeoJSON comarques
S'importa un CSV amb el nom de les comarques i el seu codi identificatiu.

In [158]:
comarques_id = pd.read_csv("comarques_codi.csv").drop(columns=["Unnamed: 3"])
comarques_id = comarques_id[(comarques_id["Nivell"] == "Comarca") | (comarques_id["Nivell"] == "Ent. terr. sing.")].drop(columns=["Nivell"]).rename(columns={"Nom": "comarca"})
comarques_id

,Codi,comarca
0,1,Alt Camp
24,2,Alt Empordà
93,3,Alt Penedès
121,4,Alt Urgell
141,5,Alta Ribagorça
145,6,Anoia
179,7,Bages
210,8,Baix Camp
239,9,Baix Ebre
254,10,Baix Empordà


Es fusionen els polígons municipals per obtenir els polígons de comarca. Després, es filtren les comarques segons si es troben a `TOTAL_COMARQUES` i se'ls hi afegeix la id de comarca.

In [162]:
comarques = df.dissolve(by=["comarca", "territori"]).reset_index().rename(columns={"territori": 'territori_id'})
comarques = pd.merge(comarques, comarques_id, on="comarca", how="outer").rename(columns={"Codi": "comarca_id"})
comarques["comarca_id"] = comarques["comarca_id"].fillna(0).astype("int")
conditions = [(comarques["territori_id"] == "CAT"), (comarques["territori_id"] == "AND"), (comarques["territori_id"] == "FRANJA"), (comarques["territori_id"] == "NORD")]
values = ["Catalunya", "Andorra", "La Franja", "Catalunya Nord"]
comarques["territori"] = np.select(conditions, values)
comarques = comarques[comarques["comarca"].isin(TOTAL_COMARQUES)]

comarques

,comarca,territori_id,geometry,vegueria,comarca_id,territori
0,Alt Camp,CAT,"POLYGON ((1.23226 41.21750, 1.22732 41.21876, ...",Camp,1,Catalunya
1,Alt Empordà,CAT,"POLYGON ((2.89860 42.12038, 2.90117 42.13878, ...",Costa Brava,2,Catalunya
2,Alt Penedès,CAT,"POLYGON ((1.62169 41.22335, 1.60414 41.23419, ...",Penedès,3,Catalunya
3,Alt Urgell,CAT,"MULTIPOLYGON (((1.31784 41.98191, 1.31277 41.9...",Alt Segre,4,Catalunya
4,Alta Ribagorça,CAT,"POLYGON ((0.87384 42.29883, 0.84891 42.31363, ...",Ribagorça i Pallars,5,Catalunya
5,Alta Ribagorça,FRANJA,"POLYGON ((0.44056 42.34615, 0.44112 42.34648, ...",Ribagorça i Pallars,5,La Franja
6,Andorra,AND,"POLYGON ((1.55703 42.45394, 1.55970 42.44832, ...",Alt Segre,0,Andorra
7,Anoia,CAT,"POLYGON ((1.46873 41.44372, 1.48561 41.46080, ...",Penedès,6,Catalunya
8,Aran,CAT,"POLYGON ((0.67298 42.71891, 0.68001 42.72318, ...",Aran,39,Catalunya
9,Bages,CAT,"POLYGON ((1.80571 41.60474, 1.78646 41.61234, ...",Llobregat,7,Catalunya


S'exporta en format GeoJSON. **Les comarques que no són de Catalunya (Cat. Nord i Andorra) tenen una id de 0, que després es canvia manualment al GeoJSON afegint números a partir de 42.** L'id assignat és arbitrari però es farà servir en totes les operacions internes de l'app, de manera que ha de ser consistent.

In [163]:
with open("comarques_parsed.geojson", "w") as f:
    f.write(comarques.to_json(drop_id=True))

## GeoJSON vegueries
Es duu a terme el mateix procés que amb les comarques però agrupant per vegueries. Es crea un id aleatori del 0 al 13 i s'exporta com a GeoJSON.

In [67]:
vegueries = df.dissolve(by="vegueria").reset_index().sort_values(by=["territori", "vegueria"]).reset_index().rename(
    columns={"index": "vegueria_id"}).drop(columns=["comarca"])
vegueries

,vegueria_id,vegueria,geometry,territori
0,1,Aran,"POLYGON ((0.67298 42.71891, 0.68001 42.72318, ...",CAT
1,2,Barcelona,"POLYGON ((1.93724 41.27449, 1.94296 41.28029, ...",CAT
2,3,Camp,"POLYGON ((0.89084 40.96679, 0.88082 40.95995, ...",CAT
3,5,Costa Brava,"POLYGON ((2.61409 41.69234, 2.59962 41.68386, ...",CAT
4,6,Ebre,"POLYGON ((0.11519 40.72326, 0.11055 40.72288, ...",CAT
5,7,Llobregat,"POLYGON ((1.43094 41.62940, 1.42176 41.63465, ...",CAT
6,8,Maresme,"POLYGON ((2.29713 41.47249, 2.29586 41.47515, ...",CAT
7,9,Penedès,"POLYGON ((1.47643 41.20892, 1.47448 41.21888, ...",CAT
8,10,Ponent,"POLYGON ((0.36815 41.29529, 0.36819 41.29462, ...",CAT
9,12,Ter-Garrotxa,"MULTIPOLYGON (((2.07477 41.90002, 2.06436 41.9...",CAT


In [85]:
with open("vegueries.geojson", "w") as f:
    f.write(vegueries.to_json(drop_id=True))

In [182]:
## S'actualitza el GeoJSON de comarques per incloure l'id de les vegueries.

comarques = gpd.read_file("comarques_parsed.geojson")
comarques = pd.merge(comarques, vegueries, on=["vegueria"], how="outer").drop(columns=["geometry_y", "territori_y"]).rename(columns={"territori_x": "territori", "geometry_x": "geometry"})
with open("comarques_parsed.geojson", "w") as f:
    f.write(comarques.to_json(drop_id=True))

## 100 cims topo
S'afegeix la informació topogràfica (territori, vegueria, comarca) al JSON de 100 cims.

In [183]:
cent_cims_df = pd.read_json("100cims.json")

comarques_cent_cims = cent_cims_df["comarca"].to_list()

cent_cims_df["territory"] = [[comarques[comarques["comarca"] == item]["territori"].iloc[0] for item in comarca] for comarca in comarques_cent_cims]
cent_cims_df["territory_id"] = [[comarques[comarques["comarca"] == item]["territori_id"].iloc[0] for item in comarca] for comarca in comarques_cent_cims]
cent_cims_df["comarca_id"] = [[comarques[comarques["comarca"] == item]["comarca_id"].iloc[0] for item in comarca] for comarca in comarques_cent_cims]
cent_cims_df["vegueria"] = [[comarques[comarques["comarca"] == item]["vegueria"].iloc[0] for item in comarca] for comarca in comarques_cent_cims]
cent_cims_df["vegueria"] = [[comarques[comarques["comarca"] == item]["vegueria"].iloc[0] for item in comarca] for comarca in comarques_cent_cims]
cent_cims_df["vegueria_id"] = [[comarques[comarques["comarca"] == item]["vegueria_id"].iloc[0] for item in comarca] for comarca in comarques_cent_cims]

In [204]:
cent_cims_df.to_json("100cims_2.json", orient="records")

In [141]:
comarques_cent_cims

[['Alt Camp'],
 ['Conflent', 'Vallespir'],
 ['Rosselló'],
 ['Alta Cerdanya'],
 ['Conflent'],
 ['Alta Cerdanya'],
 ['Capcir', 'Alta Cerdanya'],
 ['Conflent'],
 ['Alta Cerdanya', 'Conflent'],
 ['Andorra'],
 ['Andorra'],
 ['Andorra'],
 ['Vallès Oriental'],
 ['Andorra', 'Alta Cerdanya'],
 ['Vallès Occidental'],
 ['Vallès Oriental'],
 ['Andorra'],
 ['Alta Cerdanya', 'Ripollès'],
 ['Vallès Occidental'],
 ['Solsonès'],
 ['Vallès Occidental'],
 ['Selva', 'Vallès Oriental'],
 ["Val d'Aran"],
 ["Val d'Aran"],
 ["Val d'Aran"],
 ["Val d'Aran"],
 ['Segrià'],
 ['Urgell'],
 ['Vallès Occidental'],
 ['Solsonès'],
 ['Terra Alta'],
 ['Selva'],
 ['Terra Alta'],
 ['Ripollès'],
 ['Segarra'],
 ["Val d'Aran"],
 ["Val d'Aran"],
 ['Selva'],
 ['Ripollès'],
 ['Conflent', 'Ripollès'],
 ['Ripollès'],
 ['Ripollès', 'Vallespir'],
 ["Pla d'Urgell"],
 ["Ribera d'Ebre"],
 ["Ribera d'Ebre"],
 ["Ribera d'Ebre"],
 ['Priorat'],
 ["Pla de l'Estany"],
 ['Pallars Sobirà'],
 ['Pallars Sobirà', "Val d'Aran"],
 ['Pallars Sobirà']

Es genera el JSON amb l'estructura definitiva (que després s'anirà ampliant).

In [209]:
with open("100cims_2.json") as f:
    cent_cims = json.load(f)
cent_cims2 = []

for cim in cent_cims:
    cim2 = {
        "name": cim["nom"],
        "id": cim["id"],
        "coords": {
            "latlng": {
                "lat": cim["lat"],
                "lng": cim["lng"]
            },
            "utm": {
                "x": cim["utm_x"],
                "y": cim["utm_y"]
            }
        },
        "topography": {
            "territory": [{"name": x, "id": y} for x, y in zip(cim["territory"], cim["territory_id"])],
            "vegueria": [{"name": x, "id": y} for x, y in zip(cim["vegueria"], cim["vegueria_id"])],
            "comarca": [{"name": x, "id": y} for x, y in zip(cim["comarca"], cim["comarca_id"])]
        },
        "essential": cim["essencial"],
        "url": {
            "feec": cim["url"],
            "wikidata": cim["wikidata"]
        },
        "images": [] if cim["image"] is None else [cim["image"]],
        "height": cim["altitud"]
    }
    cent_cims2.append(cim2)

with open("100cims_2.json", "w") as f:
    f.write(json.dumps(cent_cims2, ensure_ascii=False, indent=4))